# Agent Orchestration with a Planner

In this workshop, we will use the planner to design a multi-agent **FLEET** designated for the task of generating a cybersecurity report. The following agents will be included:

1. time_keeper
2. cyber_collector
3. db_reader
4. data_analyzer
5. security_evaluator
6. report_writer

In [ ]:
import os
import logging
from dotenv import load_dotenv


logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(
    logging.WARNING)

load_dotenv()
print(os.getenv('CHAT_MODEL'))

# List all agents

In [3]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

In [ ]:
fleet_name = "internet_threat_analysis"
agent_fleet = []
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    if "group" in _agent.metadata.keys() and _agent.metadata["group"] == fleet_name:
        agent_fleet.append({"id": _agent.id, 
                            "name": _agent.name,
                            "description": _agent.description})
        
agent_fleet

In [5]:
aoai_client = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01")

deployment_name = os.environ["CHAT_MODEL"]

In [6]:
sys_prompt = """You are an intelligent assistant capable of responding to user queries. When you receive a message from the user, you need to determine whether you can answer it directly or if it would be more appropriate to allocate one of the listed agents to respond.

Direct Response: If the question falls within your knowledge and capabilities, you will respond directly. Your response should start with the prefix [__PLANNER__], followed by your answer.

Agent Allocation: If the query is better suited for one of the agents, you will allocate the most appropriate agent based on the user’s message. Your response will begin with the prefix [__AGENT__], followed by the agent's name. If the allocated agent is unable to provide a satisfactory answer, you may decide to allocate a different agent that may better address the user's needs.

Here is the list of agents you will be working with:
[__AGENT_LIST_PLACEHOLDER__]

Make sure to consider the descriptions of each agent when deciding which one to allocate. If an agent cannot find sufficient information or provide a relevant response, you may allocate another agent to ensure the user receives the most relevant and informed answer. Your goal is to provide clear and helpful information to the user, whether through your own knowledge or via the correct agent.

## Constraints
- Do not allocate an agent if you can answer the question directly.
- If there is an agent that can possibly answer the question, allocate that agent.
- If an agent cannot find sufficient information or provide a relevant response, you may allocate another agent to ensure the user receives the most relevant and informed answer.
- If you don't allocate an agent, you must respond with the prefix [__PLANNER__], followed by your response. For example, [__PLANNER__]Hello, I'm here to assist you.;
- If you allocate an agent, you must respond with the prefix [__AGENT__], followed by the agent's name, and then [__TASK__] followed by the request to that agent. For example: [__AGENT__]HR Helpdesk;[__TASK__]Get the company benefit policies.;
- If you think the user's question is fully answered, respond with the prefix [__TERMINATION__] followed by your response.
"""

In [ ]:
import json

cyber_fleet_str = json.dumps(agent_fleet).replace("{", '{{').replace("}", '}}')
print(sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str))

In [8]:
addtional_constraint = '''
## Guidelines to generate a report
1. Start with the time keeper to get the current time.
2. Allocate the cyber collector to retrieve latest threat information.
3. Use the metric types from the cyber collector to have the db reader fetch database data.
4. Use the CSV files from the db reader to have the data analyzer perform analysis, including forecasting and anomaly detection.
5. Allocate the security evaluator to create radar chart of evaluation metrics.
6. Start report writer to compile information and generate PDF report.
'''

In [26]:
from user_functions import user_functions
from azure.ai.projects.models import FunctionTool, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

functions = FunctionTool(functions=user_functions)


def agent_execution(agent_id, task, context):

    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id, role="user", content=f'task: {task} \n\n context: {context}')

    print(f"Created message, ID: {message.id}")

    # Create and process assistant run in thread with tools
    run = project_client.agents.create_run(
        thread_id=thread.id, assistant_id=agent_id)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.cancel_run(
                    thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.agents.submit_tool_outputs_to_run(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

    messages = project_client.agents.list_messages(thread_id=thread.id)
    return messages

In [27]:
def get_assistant_content(agent_messages):
    contents = []
    for message in agent_messages:
        if message['role'] == 'assistant':
            contents.append(message['content'][0]['text']['value'])
    return contents

In [35]:
messages = [
    {
        "role": "system",
        "content": sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str) + addtional_constraint
    },
    {
        "role": "user",
        "content": "Generate a cybersecurity report for TrendMicro."
    },
]

In [ ]:
messages[1:]

In [ ]:
response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    temperature=0.7,
    max_tokens=1000
)

llm_response = response.choices[0].message.content

messages.append({"role": "assistant", "content": f"{llm_response}"})
while "[__AGENT__]" in llm_response:
    agent_name = llm_response.split("[__AGENT__]")[1].split(";")[0]
    print(f"Allocated agent: {agent_name}")
    agent_task = llm_response.split("[__TASK__]")[1]
    print(f"Agent task: {agent_task}")
    # find agent id from agent fleet based on agent_name
    agent_id = None
    for agent in agent_fleet:
        if agent["name"] == agent_name:
            agent_id = agent["id"]
            break

    if agent_id:
        agent_messages = agent_execution(agent_id, agent_task, json.dumps(messages[1:]))
        agent_contents = get_assistant_content(agent_messages.data)
        if agent_contents:
            for agent_content in agent_contents:
                messages.append(
                    {"role": "assistant", "content": f"[__AGENT__({agent_name})]{agent_content}"})

        response = aoai_client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            temperature=0.7,
            max_tokens=1000
        )
        llm_response = response.choices[0].message.content
        messages.append({"role": "assistant", "content": f"{llm_response}"})
    else:
        break

print(messages)

In [ ]:
messages = [
    {
        "role": "system",
        "content": sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str) + addtional_constraint
    },
    {
        "role": "user",
        "content": "Generate a cybersecurity report for TrendMicro."
    },
]

response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    temperature=0.7,
    max_tokens=1000
)
messages.append({"role": "assistant", "content": f"{llm_response}"})

llm_response = response.choices[0].message.content
if "[__AGENT__]" in llm_response:
    agent_name = llm_response.split("[__AGENT__]")[1].split(";")[0]
    print(f"Allocated agent: {agent_name}")
    agent_task = llm_response.split("[__TASK__]")[1]
    print(f"Agent task: {agent_task}")
    # find agent id from agent fleet based on agent_name
    agent_id = None
    for agent in agent_fleet:
        if agent["name"] == agent_name:
            agent_id = agent["id"]
            break

    if agent_id:
        agent_messages = agent_execution(agent_id, agent_task)
        messages.append({"role": "assistant", "content": f"[__AGENT__({agent_name})]{get_assistant_content(agent_messages.data)}"})
        print(messages)

        response = aoai_client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            temperature=0.7,
            max_tokens=1000
        )

        llm_response = response.choices[0].message.content
        messages.append({"role": "assistant", "content": f"{llm_response}"})
        print(llm_response)
        
        if "[__AGENT__]" in llm_response:
            agent_name = llm_response.split("[__AGENT__]")[1].split(";")[0]
            print(f"Allocated agent: {agent_name}")
            agent_task = llm_response.split("[__TASK__]")[1]
            print(f"Agent task: {agent_task}")
            # find agent id from agent fleet based on agent_name
            agent_id = None
            for agent in agent_fleet:
                if agent["name"] == agent_name:
                    agent_id = agent["id"]
                    break

            if agent_id:
                agent_messages = agent_execution(agent_id, agent_task)
                messages.append({"role": "assistant", "content": f"[__AGENT__({agent_name})]{get_assistant_content(agent_messages.data)}"})
                
                response = aoai_client.chat.completions.create(
                    model=deployment_name,
                    messages=messages,
                    temperature=0.7,
                    max_tokens=1000
                )

                llm_response = response.choices[0].message.content
                messages.append({"role": "assistant", "content": f"{llm_response}"})
                
                if "[__AGENT__]" in llm_response:
                    agent_name = llm_response.split("[__AGENT__]")[1].split(";")[0]
                    print(f"Allocated agent: {agent_name}")
                    agent_task = llm_response.split("[__TASK__]")[1]
                    print(f"Agent task: {agent_task}")
                    # find agent id from agent fleet based on agent_name
                    agent_id = None
                    for agent in agent_fleet:
                        if agent["name"] == agent_name:
                            agent_id = agent["id"]
                            break

                    if agent_id:
                        agent_messages = agent_execution(agent_id, agent_task)
                        messages.append({"role": "assistant", "content": f"[__AGENT__({agent_name})]{get_assistant_content(agent_messages.data)}"})
                        
                        response = aoai_client.chat.completions.create(
                            model=deployment_name,
                            messages=messages,
                            temperature=0.7,
                            max_tokens=1000
                        )

                        llm_response = response.choices[0].message.content
                        messages.append({"role": "assistant", "content": f"{llm_response}"})

                        if "[__AGENT__]" in llm_response:
                            agent_name = llm_response.split("[__AGENT__]")[1].split(";")[0]
                            print(f"Allocated agent: {agent_name}")
                            agent_task = llm_response.split("[__TASK__]")[1]
                            print(f"Agent task: {agent_task}")
                            # find agent id from agent fleet based on agent_name
                            agent_id = None
                            for agent in agent_fleet:
                                if agent["name"] == agent_name:
                                    agent_id = agent["id"]
                                    break

                            if agent_id:
                                agent_messages = agent_execution(agent_id, agent_task)
                                messages.append({"role": "assistant", "content": f"[__AGENT__({agent_name})]{get_assistant_content(agent_messages.data)}"})
                                
                                response = aoai_client.chat.completions.create(
                                    model=deployment_name,
                                    messages=messages,
                                    temperature=0.7,
                                    max_tokens=1000
                                )

                                llm_response = response.choices[0].message.content
                                messages.append({"role": "assistant", "content": f"{llm_response}"})
                        

elif "[__TERMINATION__]" in llm_response:
    print("Direct response")
    
print(messages)

In [ ]:
print(agent_messages['data'])